In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as nnfunc
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim import SGD
from torch.optim import lr_scheduler
import torchvision
import glob
from torch.utils.data import random_split, Subset

ModuleNotFoundError: No module named 'torch'

In [ ]:
# TODOs

# Data preprocessing:
# TODO remove color (grey scale)
# TODO remove background maybe?

# Optimizer
# TODO Adam vs. SGD?

# Model
# What's unique for encrypted image classification on top of normal class.?

In [ ]:
classes = ['f','j','k','l','m','n','o','x','y','z']

transformer = transforms.Compose([
    transforms.ToTensor(),
#     transforms.RandomHorizontalFlip(),
    transforms.Normalize([0.5,0.5,0.5],
                        [0.5,0.5,0.5])
])

# TODO remove color (grey scale)
# TODO remove background maybe?

train_path = 'data_letters/train'
test_path = 'data_letters/test'

train_data = torchvision.datasets.ImageFolder(train_path, transform=transformer)

In [ ]:
train_ratio = 0.8
train_size = int(train_ratio * len(train_data))
val_size = len(train_data) - train_size
train_set, val_set = random_split(train_data, [train_size, val_size])
print(len(train_set))
print(len(val_set))

In [ ]:
trainloader = DataLoader(
    train_set,
    batch_size = 256,
    shuffle = True
)

valloader = DataLoader(
    val_set,
    batch_size = 256,
    shuffle = True
)

In [ ]:
# Own ResNet model
class Task1Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Task1Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        self.relu1 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.dropout = nn.Dropout(0.5)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.conv3 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        self.relu3 = nn.ReLU()
        self.fc = nn.Linear(in_features=32*32*32, out_features=num_classes)
        
    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        output = self.pool(output)
        output = self.dropout(output)
        output = self.conv2(output)
        output = self.relu2(output)
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        output = output.view(-1, 32*32*32)
        output = self.fc(output)
        return output
    
class Task1Res(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super().__init__()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# New training
model = torchvision.models.resnet34(pretrained=False)

inchannel = model.fc.in_features
model.fc = nn.Linear(inchannel, 10)

# Continue training - load previous model
# best_model = torch.load('model/task1.model')
# model.load_state_dict(best_model, strict=False)

model.to(device)
torch.save(model.state_dict(), 'model/task1-34-no-pretrain.model')

# TODO: Optimizers SGD vs. Adam
optimizer = Adam(model.parameters(), lr=0.001)
# optimizer = SGD(model.parameters(), lr=0.001)
# optimizer = SGD(model.parameters(), lr=0.0001, weight_decay=5e-4)
loss_function = nn.CrossEntropyLoss()

num_epoches = 30

train_count = len(train_set)
val_count = len(val_set)

best_accuracy = 0.0

hist = []

for epoch in range(num_epoches):
    model.to(device)
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0
    for i, (images, labels) in enumerate(trainloader):
        if torch.cuda.is_available():
            images = images.to(device)
            labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.cpu().data * images.size(0)
        _, prediction = torch.max(outputs.data,1)
        train_accuracy += int(torch.sum(prediction == labels.data))
    
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss/train_count
    print(f"Epoch {epoch}")
    print(f"Train_accuracy {train_accuracy}")
    
    model.eval()
    val_accuracy = 0.0
    with torch.no_grad():
        for i, (images, labels) in enumerate(valloader):
            if torch.cuda.is_available():
                images = images.to(device)
                labels = labels.to(device)
            outputs = model(images)
            _,prediction = torch.max(outputs.data,1)
            val_accuracy += int(torch.sum(prediction==labels.data))

        val_accuracy = val_accuracy / val_count
        print(f"Train Loss {train_loss}")
        print(f"Val Accuracy {val_accuracy}")
        
        hist.append(val_accuracy)
        if val_accuracy > best_accuracy:
            print(f"Best Val Accuracy {val_accuracy}")
            torch.save(model.state_dict(), 'model/task1-34-no-pretrain.model')
            best_accuracy = val_accuracy

In [ ]:
hist